# Confusion Matrix

For $N$ samples, want confusion matrix that looks like:

In [1]:
import pandas as pd
dummy_confusion_matrix = pd.DataFrame()

dummy_confusion_matrix['labelled_pos'] = ['a',0]
dummy_confusion_matrix['labelled_neg'] = [0,'b']

dummy_confusion_matrix['unlabelled'] = ['c','d']

dummy_confusion_matrix.index = ['predicted_pos','predicted_neg']

print(dummy_confusion_matrix)


              labelled_pos labelled_neg unlabelled
predicted_pos            a            0          c
predicted_neg            0            b          d


Call the fraction of all positive samples $\alpha$. Then $$(1)\quad\frac{a+c}{N} = \alpha, \qquad \frac{b+d}{N}=1-\alpha\quad(2)$$

Call the fraction of all labelled samples $\beta$. Then $$(3)\quad\frac{a+b}{N}=\beta,\qquad\frac{c+d}{N}=1-\beta\quad(4)$$

Call the fraction of labelled positives (out of all labelled samples) $\gamma$. Then $$\frac{a}{a+b}=\gamma\quad(5)$$

$\beta$ and $\gamma$ are chosen by us (for now, we'll use $\beta=0.01$ and $\gamma=0.4$ as defaults). $\alpha$ is found from the data.

We can rearrange these to give...

$$a = \beta\gamma N\\ b=\beta(1-\gamma)N\\ c=(\alpha-\beta\gamma)N\\ d=(1-\alpha-\beta+\beta\gamma)N$$

# Labelled Sample Selection

`remove_labels` (in `preprocess.py`) takes a dataframe ($N$ rows/samples) with a `fraud_bool` Boolean column. It creates a `fraud_masked` column, where some samples retain their labels and others are de-labelled (set to NaN).

Of all positive samples, $a$ are kept labelled and the remaining $c$ are de-labelled. Likewise of all negative samples, $b$ are kept labelled and the remaining $d$ are de-labelled.

There are 3 ways `remove_labels` can choose which samples to label/de-label:
* Uniformly at random (`probs` is None)
    - Of all $a+c$ positive samples, $a$ are chosen uniformly at random (using `numpy.random.choice` with `p=None`)
    - Likewise for negative samples
* At random, with weighted proability (`probs` is an iterable of floats, top_probs_only is `False`)
    - A `probs` vector is supplied, specifying relative probabilities for each sample (to remain labelled)
    - Across all positive samples, the corresponding `probs` values are normalised, and $a$ samples are selected with these normalised probabilities
    - i.e. a positive sample with 0.8 in `probs` is twice as likely to remain labelled as a positive sample with `0.4`
    - Likewise for negative samples
* Deterministically according to a ranking (`probs` is an iterable of floats, `top_probs_only` is `True`)
    - A `probs` vector is supplied
    - Across all positive samples, those with the $a$ largest values in `probs` remain labelled
    - Likewise for negative samples

# Weak learner for initial label selection

If used, `probs` is intended to be the predicted fraud (positive-class) probability, output by some existing model. To mimic the case where the initially-labelled data will have been flagged as suspicious, we first make a simple model, then use its predictions to de-label the data.

We use a (regularised) logistic regression (found in `src/preprocess_weights.py`)

Cohen's $\kappa\approx0.06$ suggesting only marginally better performance than a random classifier with the same overall class proportions. F1 $\approx0.08$, and recall at 5% FPR $\approx0.50$. This is a poor model, but is hopefully good enough that the most 'suspicious' samples (by model prediction) are a good representation of those that would be flagged in a real-world scenario.

(When trained on only the training split, and evaluated on test/valid, all metrics were same as reported above, to within 0.01)

The predicted class and probabilities for the training data are saved in `src/preprocess/weaklearner_weights.csv`. They can be incorporated in de-labelling by imporFor both train and test: ting into `src/preprocess.py` (as described in the script).